In [1]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_cores=2
launcher.executor_memory='2000m'
launcher.packages=["com.github.master:spark-stemming_2.10:0.2.0"]


In [2]:
//Read the CSV file and load it into a dataframe.the "inferschema" parameter is set to true
val creditCard_df=spark.read.option("header","true").option("inferschema", "true").csv("/hadoop-user/data/creditcard.csv")
creditCard_df.cache()
creditCard_df.printSchema()
creditCard_df.show(3)
creditCard_df.count
//creditCard_df.createOrReplaceTempView("creditCard_df")


Intitializing Scala interpreter ...

Spark Web UI available at http://bd-hm04:8088/proxy/application_1544314460278_0001
SparkContext available as 'sc' (version = 2.3.2, master = yarn, app id = application_1544314460278_0001)
SparkSession available as 'spark'


2018-12-08 18:16:29 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
root
 |-- Time: decimal(10,0) (nullable = true)
 |-- V1: double (nullable = true)
 |-- V2: double (nullable = true)
 |-- V3: double (nullable = true)
 |-- V4: double (nullable = true)
 |-- V5: double (nullable = true)
 |-- V6: double (nullable = true)
 |-- V7: double (nullable = true)
 |-- V8: double (nullable = true)
 |-- V9: double (nullable = true)
 |-- V10: double (nullable = true)
 |-- V11: double (nullable = true)
 |-- V12: double (nullable = true)
 |-- V13: double (nullable = true)
 |-- V14: double (nullable = true)
 |-- V15: double (nullable = true)
 |-- V16: double (nullable = true)
 |-- V17: double (nullable = true)
 |-- V18: double (nullable = true)
 |-- V19: double (nullable = true)
 |-- V20: double (nullable = true)
 |-- V21: double (nullable = true)
 |-- V22: double (nullable =

creditCard_df: org.apache.spark.sql.DataFrame = [Time: decimal(10,0), V1: double ... 29 more fields]
res0: Long = 284807


In [192]:
// to check for null values in our dataset
creditCard_df.na.drop
creditCard_df.count()


res69: Long = 284807


In [ ]:
//get idea of mean, mode,st deviation
creditCard_df.describe().show(9)

In [87]:
import org.apache.spark.ml.feature._
//vector assembler to assemble these columns to a single feature vector.
val assembler=new VectorAssembler().setInputCols(Array("Amount","Time")).setOutputCol("feature_vector")

//val feature_vector=assembler.transform(creditCard_df).select("feature_vector").toDF("creditCard_df").show()



+-------------+
|creditCard_df|
+-------------+
| [149.62,0.0]|
|   [2.69,0.0]|
| [378.66,1.0]|
|  [123.5,1.0]|
|  [69.99,2.0]|
|   [3.67,2.0]|
|   [4.99,4.0]|
|   [40.8,7.0]|
|   [93.2,7.0]|
|   [3.68,9.0]|
|   [7.8,10.0]|
|  [9.99,10.0]|
| [121.5,10.0]|
|  [27.5,11.0]|
|  [58.8,12.0]|
| [15.99,12.0]|
| [12.99,12.0]|
|  [0.89,13.0]|
|  [46.8,14.0]|
|   [5.0,15.0]|
+-------------+
only showing top 20 rows



import org.apache.spark.ml.feature._
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_b62ec09eb19e
feature_vector: Unit = ()


In [163]:

import org.apache.spark.ml.feature._
import org.apache.spark.ml.linalg._

val standardized=new StandardScaler().setWithMean(true).setInputCol("feature_vector").setOutputCol("scaled_and_centered")
println(standardized)

stdScal_e5d9a5664efc


import org.apache.spark.ml.feature._
import org.apache.spark.ml.linalg._
standardized: org.apache.spark.ml.feature.StandardScaler = stdScal_e5d9a5664efc


In [94]:
// another vector assembler to assemble remaining columns to a single feature vector.
val assembler1=new VectorAssembler().setInputCols(Array("scaled_and_centered","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28")).setOutputCol("feature_vector1")





assembler1: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_0dbb091556d8


In [113]:
//MinMaxScaler estimator which takes a column of feature vectors and computes its min-max scaling
val minmax=new MinMaxScaler().setInputCol("feature_vector1").setOutputCol("min_max_scaled")


minmax: org.apache.spark.ml.feature.MinMaxScaler = minMaxScal_0df5b5e00432


In [7]:
// Getting count of fraud and non-fraud class
val num=sql("select Class from creditCard_df where Class= 1").count()
val num1=sql("select Class from creditCard_df where Class= 0").count()
val total_count=creditCard_df.count()

num: Long = 492
num1: Long = 284315
total_count: Long = 284807


In [172]:
// using weighted data(under-sampling) to balance the data
val balance_ratio=(total_count-num).toDouble / total_count
println(balance_ratio)
val calculateWeight= udf { d: Double =>
if (d == 0.0) {
    1 * balance_ratio
}
    else{
        (1 * (1.0 - balance_ratio))
    }
}
// cresting new dataset with balanced data
val weightedDataset = creditCard_df.withColumn("classWeightCol", calculateWeight(creditCard_df("Class")))




0.9982725143693799


balance_ratio: Double = 0.9982725143693799
calculateWeight: org.apache.spark.sql.expressions.UserDefinedFunction = UserDefinedFunction(<function1>,DoubleType,Some(List(DoubleType)))
weightedDataset: org.apache.spark.sql.DataFrame = [Time: decimal(10,0), V1: double ... 30 more fields]


In [173]:
//create a logistic regression model using LogisticRegression class in spark and set its input column to the classWeightCol and its output column to the user defined Class
val lr_w= new LogisticRegression().setWeightCol("classWeightCol").setLabelCol("Class").setFeaturesCol("feature_vector")

lr_w: org.apache.spark.ml.classification.LogisticRegression = logreg_b826b840105c


In [174]:
val pipeline_w = new Pipeline().setStages(Array(assembler,standardized,lr_w))

pipeline_w: org.apache.spark.ml.Pipeline = pipeline_99719d8736fb


In [175]:
val Array(training_w,testing_w)=weightedDataset.randomSplit(Array(0.8,0.2),111)

training_w: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 30 more fields]
testing_w: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 30 more fields]


In [176]:
//Fit the training data to the pipeline
val pipelineModel_w = pipeline_w.fit(training_w)

pipelineModel_w: org.apache.spark.ml.PipelineModel = pipeline_99719d8736fb


In [161]:
// Make predictions.
val predictions_w = pipelineModel_w.transform(testing_w)


predictions_w: org.apache.spark.sql.DataFrame = [Time: decimal(10,0), V1: double ... 35 more fields]


In [179]:
val paramGrid_w =new ParamGridBuilder()
             .addGrid(lr_w.regParam, Array(0.01, 0.5, 2.0))
             .addGrid(lr_w.elasticNetParam, Array(0.0, 0.5, 1.0))
             .build()
//evaluate the model with the test data.
val evaluator_w = new BinaryClassificationEvaluator().setRawPredictionCol("rawPrediction").setLabelCol("Class").setMetricName("areaUnderROC")
//a 5 fold cross validation is used to tune hyper-parameters
val cv_w = new CrossValidator().setEstimator(lr_w).setEvaluator(evaluator_w).setEstimatorParamMaps(paramGrid_w).setNumFolds(5)


//// Select example rows to display.
predictions_w.select("prediction","Class","probability").show()
val AUC_w = evaluator_w.evaluate(predictions_w)
println(s"Area under ROC curve(AUC) for RF on test data = $AUC_w")

+----------+-----+--------------------+
|prediction|Class|         probability|
+----------+-----+--------------------+
|       0.0|    0|[0.99999446344923...|
|       0.0|    0|[0.99999433352264...|
|       0.0|    0|[0.99999443769908...|
|       0.0|    0|[0.99999440327654...|
|       0.0|    0|[0.99999444317901...|
|       0.0|    0|[0.99999446725021...|
|       0.0|    0|[0.99999442235565...|
|       0.0|    0|[0.99999446644778...|
|       0.0|    0|[0.99999445119999...|
|       0.0|    0|[0.99999444660485...|
|       0.0|    0|[0.99999270452405...|
|       0.0|    0|[0.99999444503214...|
|       0.0|    0|[0.99999433526159...|
|       0.0|    0|[0.99999443338983...|
|       0.0|    0|[0.99999440339310...|
|       0.0|    0|[0.99999431936653...|
|       0.0|    0|[0.99999444741057...|
|       0.0|    0|[0.99999446158170...|
|       0.0|    0|[0.99999446918852...|
|       0.0|    0|[0.99999445427104...|
+----------+-----+--------------------+
only showing top 20 rows

Area under ROC

paramGrid_w: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_b826b840105c-elasticNetParam: 0.0,
	logreg_b826b840105c-regParam: 0.01
}, {
	logreg_b826b840105c-elasticNetParam: 0.5,
	logreg_b826b840105c-regParam: 0.01
}, {
	logreg_b826b840105c-elasticNetParam: 1.0,
	logreg_b826b840105c-regParam: 0.01
}, {
	logreg_b826b840105c-elasticNetParam: 0.0,
	logreg_b826b840105c-regParam: 0.5
}, {
	logreg_b826b840105c-elasticNetParam: 0.5,
	logreg_b826b840105c-regParam: 0.5
}, {
	logreg_b826b840105c-elasticNetParam: 1.0,
	logreg_b826b840105c-regParam: 0.5
}, {
	logreg_b826b840105c-elasticNetParam: 0.0,
	logreg_b826b840105c-regParam: 2.0
}, {
	logreg_b826b840105c-elasticNetParam: 0.5,
	logreg_b826b840105c-regParam: 2.0
}, {
	logreg_b826b840105c-elasticNetParam: 1.0,
	logreg_b826b840105c-r...

In [71]:
//displaying count of target variable
val count = creditCard_df.groupBy("Class").count().show()

+-----+------+
|Class| count|
+-----+------+
|    1|   492|
|    0|284315|
+-----+------+



count: Unit = ()


In [184]:
//mapping the data to balance
val fractionKeyMap1= Map(1.0->1.0, 0.0->0.002)
//stat.sampleBy used to sample down the data
val sampling= creditCard_df.stat.sampleBy("Class",fractionKeyMap1, 111)
//.groupBy("Class").count().show()

fractionKeyMap1: scala.collection.immutable.Map[Double,Double] = Map(1.0 -> 1.0, 0.0 -> 0.002)
sampling: org.apache.spark.sql.DataFrame = [Time: decimal(10,0), V1: double ... 29 more fields]


In [37]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature._
//create a Random Forest model using RandomForestClassifier in spark to predict the Class for this dataset.
val rf1 = new RandomForestClassifier().setLabelCol("Class").setFeaturesCol("feature_vector1")

import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.feature._
rf1: org.apache.spark.ml.classification.RandomForestClassifier = rfc_8cd412573f9b


In [38]:
//fit stages in the pipeline
val pipeline_rf1 = new Pipeline().setStages(Array(assembler,standardized, assembler1,rf1))

pipeline_rf1: org.apache.spark.ml.Pipeline = pipeline_133af343a912


In [39]:
//dividing data into trainig and testing set
val Array(training1,testing1)=sampling.randomSplit(Array(0.8,0.2),111)

training1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 29 more fields]
testing1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 29 more fields]


In [40]:
//Fit the training data to the pipeline
val pipelineModel_rf1 = pipeline_rf1.fit(training1)

pipelineModel_rf1: org.apache.spark.ml.PipelineModel = pipeline_133af343a912


In [178]:
// Make predictions.
val predictions1 = pipelineModel_rf1.transform(testing1)
predictions1.select("prediction","Class","probability").show()

+----------+-----+--------------------+
|prediction|Class|         probability|
+----------+-----+--------------------+
|       0.0|    0|[0.85092936668793...|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       0.0|    0|[0.94007073557125...|
|       0.0|    0|[0.82676779657402...|
|       1.0|    1|           [0.0,1.0]|
|       0.0|    0|[0.82632182176102...|
|       0.0|    0|[0.89531684103582...|
|       1.0|    1|[0.30781249213438...|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       1.0|    1|           [0.0,1.0]|
|       0.0|    0|[0.95709351339265...|
|       1.0|    1|           [0.0,1.0]|
+----------+-----+--------------------+
only showing top 20 rows



predictions1: org.apache.spark.sql.DataFrame = [Time: decimal(10,0), V1: double ... 35 more fields]


In [177]:

 val paramGrid_rf =new ParamGridBuilder()
             .addGrid(rf.maxDepth, Array(2, 5))
             .addGrid(rf.numTrees, Array(5, 20))
             .build()

 val evaluator_rf = new BinaryClassificationEvaluator().setRawPredictionCol("rawPrediction").setLabelCol("Class").setMetricName("areaUnderROC")


val cv_rf = new CrossValidator().setEstimator(rf1).setEvaluator(evaluator_rf).setEstimatorParamMaps(paramGrid_rf).setNumFolds(2)



val AUC = evaluator.evaluate(predictions1)
predictions1.select("prediction","Class","scaled_and_centered").show()
println(s"Area under ROC curve(AUC) for RF on test data = $AUC")

+----------+-----+--------------------+
|prediction|Class| scaled_and_centered|
+----------+-----+--------------------+
|       0.0|    0|[-0.3359314193975...|
|       1.0|    1|[-0.4748082390912...|
|       1.0|    1|[-0.4748082390912...|
|       1.0|    1|[-0.4748082390912...|
|       1.0|    1|[-0.4748082390912...|
|       1.0|    1|[-0.4748082390912...|
|       1.0|    1|[-0.4748082390912...|
|       0.0|    0|[-0.1238803503823...|
|       0.0|    0|[0.12417939072971...|
|       1.0|    1|[-0.4748082390912...|
|       0.0|    0|[-0.4213977856998...|
|       0.0|    0|[-0.4763025748810...|
|       1.0|    1|[-0.4613592169827...|
|       1.0|    1|[0.43037361450429...|
|       1.0|    1|[-0.4748082390912...|
|       1.0|    1|[-0.4748082390912...|
|       1.0|    1|[0.00236692166826...|
|       1.0|    1|[0.00236692166826...|
|       0.0|    0|[-0.3272546309404...|
|       1.0|    1|[0.00236692166826...|
+----------+-----+--------------------+
only showing top 20 rows

Area under ROC

paramGrid_rf: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	rfc_75d4828f3d10-maxDepth: 2,
	rfc_75d4828f3d10-numTrees: 5
}, {
	rfc_75d4828f3d10-maxDepth: 2,
	rfc_75d4828f3d10-numTrees: 20
}, {
	rfc_75d4828f3d10-maxDepth: 5,
	rfc_75d4828f3d10-numTrees: 5
}, {
	rfc_75d4828f3d10-maxDepth: 5,
	rfc_75d4828f3d10-numTrees: 20
})
evaluator_rf: org.apache.spark.ml.evaluation.BinaryClassificationEvaluator = binEval_a80c85f984f8
cv_rf: org.apache.spark.ml.tuning.CrossValidator = cv_3a5f63ce1fc8
AUC: Double = 0.9671067352875363


In [46]:
import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}
//create a Gradient Boost Tree model using GBTClassifier in spark to predict the Class for this dataset.
val gbt = new GBTClassifier()
  .setLabelCol("Class")
  .setFeaturesCol("feature_vector1").setMaxIter(10)
//default depth of tree is 5

import org.apache.spark.ml.classification.{GBTClassificationModel, GBTClassifier}
gbt: org.apache.spark.ml.classification.GBTClassifier = gbtc_19a4a8bc5a40


In [47]:
//fitting stages to pipeline
val pipeline_gbt = new Pipeline().setStages(Array(assembler,standardized, assembler1,gbt))

pipeline_gbt: org.apache.spark.ml.Pipeline = pipeline_9d88eda9d420


In [49]:
//dividing sampled data in training and testing data
val Array(training_gbt,testing_gbt)=sampling.randomSplit(Array(0.8,0.2),111)

training_gbt: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 29 more fields]
testing_gbt: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 29 more fields]


In [50]:
val pipelineModel_gbt = pipeline_gbt.fit(training_gbt)

pipelineModel_gbt: org.apache.spark.ml.PipelineModel = pipeline_9d88eda9d420


In [180]:
//making predictions
val predictions_gbt = pipelineModel_gbt.transform(testing_gbt)
predictions_gbt.select("prediction","Class","probability").show()

+----------+-----+--------------------+
|prediction|Class|         probability|
+----------+-----+--------------------+
|       0.0|    0|[0.92963109708144...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       0.0|    0|[0.92659704687853...|
|       0.0|    0|[0.90350837796840...|
|       1.0|    1|[0.06588111931387...|
|       0.0|    0|[0.92759874330279...|
|       0.0|    0|[0.75187648812100...|
|       1.0|    1|[0.41259766059674...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       0.0|    0|[0.91310746078303...|
|       1.0|    1|[0.06588111931387...|
+----------+-----+--------------------+
only showing top 20 rows



predictions_gbt: org.apache.spark.sql.DataFrame = [Time: decimal(10,0), V1: double ... 35 more fields]


In [55]:
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
val evaluator_gbt= new MulticlassClassificationEvaluator()
.setLabelCol("Class").setPredictionCol("prediction").setMetricName("accuracy")
val Accuracy= evaluator_gbt.evaluate(predictions_gbt)
println(s"Accuracy=$Accuracy")

Accuracy=0.9137931034482759


import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
evaluator_gbt: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_a159df9080de
Accuracy: Double = 0.9137931034482759


In [60]:
import org.apache.spark.ml._
import org.apache.spark.ml.classification.LogisticRegression
//using logistc regression again to test sampled data By sampleBy method 
val lr = new LogisticRegression().setLabelCol("Class").setFeaturesCol("feature_vector1")
.setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)

import org.apache.spark.ml._
import org.apache.spark.ml.classification.LogisticRegression
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_0ae26448c1c6


In [61]:
val pipeline_lr = new Pipeline().setStages(Array(assembler,standardized, assembler1,gbt))

pipeline_lr: org.apache.spark.ml.Pipeline = pipeline_604b4ec240df


In [62]:
val Array(training_lr,testing_lr)=sampling.randomSplit(Array(0.8,0.2),111)

training_lr: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 29 more fields]
testing_lr: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Time: decimal(10,0), V1: double ... 29 more fields]


In [64]:
val pipelineModel_lr = pipeline_lr.fit(training_lr)

pipelineModel_lr: org.apache.spark.ml.PipelineModel = pipeline_604b4ec240df


In [182]:
val predictions_lr = pipelineModel_lr.transform(testing_lr)
predictions_lr.select("prediction","Class","probability").show()

+----------+-----+--------------------+
|prediction|Class|         probability|
+----------+-----+--------------------+
|       0.0|    0|[0.92963109708144...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       0.0|    0|[0.92659704687853...|
|       0.0|    0|[0.90350837796840...|
|       1.0|    1|[0.06588111931387...|
|       0.0|    0|[0.92759874330279...|
|       0.0|    0|[0.75187648812100...|
|       1.0|    1|[0.41259766059674...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       1.0|    1|[0.06588111931387...|
|       0.0|    0|[0.91310746078303...|
|       1.0|    1|[0.06588111931387...|
+----------+-----+--------------------+
only showing top 20 rows



predictions_lr: org.apache.spark.sql.DataFrame = [Time: decimal(10,0), V1: double ... 35 more fields]


In [185]:
import org.apache.spark.ml._
import org.apache.spark.ml.classification.LogisticRegression
val lr1 = new LogisticRegression().setLabelCol("Class").setFeaturesCol("feature_vector1")
.setMaxIter(10)
val paramGrid1 =new ParamGridBuilder()
             .addGrid(lr1.regParam, Array(0.01,0.5,2.0))
             .addGrid(lr1.elasticNetParam, Array(0.0,0.5,1.0))
              //.addGrid(tfidf.minDocFreq, Array(5,10))//tfidf not done, dow we need this
             .build()

val evaluator_lr1 = new BinaryClassificationEvaluator().setRawPredictionCol("rawPrediction").setLabelCol("Class").setMetricName("areaUnderROC")


val cv_lr = new CrossValidator().setEstimator(lr1).setEvaluator(evaluator_lr1).setEstimatorParamMaps(paramGrid1).setNumFolds(2)

val pipeline_lr1 = new Pipeline().setStages(Array(assembler,standardized, assembler1,cv_lr))//is pipeline ok

val Array(training_lr1,testing_lr1)=sampling.randomSplit(Array(0.8,0.2),111)

//Fit the training data to the pipeline
val pipelineModel_lr1 = pipeline_lr1.fit(training_lr1)

// Make predictions.
val predictions_lr1 = pipelineModel_lr1.transform(testing_lr1)
val AUC_lr1 = evaluator_lr1.evaluate(predictions_lr1)
predictions_lr1.select("prediction","Class","probability").show()
println(s"Area under ROC curve(AUC) for LR on test data = $AUC_lr1")

+----------+-----+--------------------+
|prediction|Class|         probability|
+----------+-----+--------------------+
|       0.0|    0|[0.95263376097556...|
|       1.0|    1|[2.00993105235670...|
|       1.0|    1|[9.54629059496377...|
|       1.0|    1|[1.12611289681051...|
|       1.0|    1|[7.16736636931522...|
|       1.0|    1|[0.00113885522413...|
|       1.0|    1|[1.05538180094523...|
|       0.0|    0|[0.92817784652995...|
|       0.0|    0|[0.80393036260579...|
|       1.0|    1|[2.90908992895979...|
|       0.0|    0|[0.95410443842753...|
|       0.0|    0|[0.97553884320673...|
|       0.0|    1|[0.65411445829338...|
|       1.0|    1|[3.16742815844637...|
|       1.0|    1|[1.14056625858585...|
|       1.0|    1|[1.28766568459304...|
|       1.0|    1|[2.73149858061735...|
|       1.0|    1|[7.23626083217651...|
|       0.0|    0|[0.93282920287423...|
|       1.0|    1|[1.84255942494730...|
+----------+-----+--------------------+
only showing top 20 rows

Area under ROC

import org.apache.spark.ml._
import org.apache.spark.ml.classification.LogisticRegression
lr1: org.apache.spark.ml.classification.LogisticRegression = logreg_aa5be670a6c1
paramGrid1: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_aa5be670a6c1-elasticNetParam: 0.0,
	logreg_aa5be670a6c1-regParam: 0.01
}, {
	logreg_aa5be670a6c1-elasticNetParam: 0.0,
	logreg_aa5be670a6c1-regParam: 0.5
}, {
	logreg_aa5be670a6c1-elasticNetParam: 0.0,
	logreg_aa5be670a6c1-regParam: 2.0
}, {
	logreg_aa5be670a6c1-elasticNetParam: 0.5,
	logreg_aa5be670a6c1-regParam: 0.01
}, {
	logreg_aa5be670a6c1-elasticNetParam: 0.5,
	logreg_aa5be670a6c1-regParam: 0.5
}, {
	logreg_aa5be670a6c1-elasticNetParam: 0.5,
	logreg_aa5be670a6c1-regParam: 2.0
}, {
	logreg_aa5be670a6c1-elasticNetParam: 1.0,
	logreg_aa5be670a6c...